In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)  # Lire en niveaux de gris
        if img is not None:
            # Redimensionner l'image pour avoir une taille uniforme
            img = cv2.resize(img, (64, 64))
            images.append(img)
    return images

In [7]:
# Charger les images de chaque classe pour l'entraînement et les tests
train_normal_images = load_images_from_folder('./data/chest_Xray/train/NORMAL')
train_malade_images = load_images_from_folder('./data/chest_Xray/train/PNEUMONIA')
test_normal_images = load_images_from_folder('./data/chest_Xray/test/NORMAL')
test_malade_images = load_images_from_folder('./data/chest_Xray/test/PNEUMONIA')

# Créer les labels correspondants
train_normal_labels = ['normal'] * len(train_normal_images)
train_malade_labels = ['malade'] * len(train_malade_images)
test_normal_labels = ['normal'] * len(test_normal_images)
test_malade_labels = ['malade'] * len(test_malade_images)

# Combiner les images et les labels
images = train_normal_images + train_malade_images + test_normal_images + test_malade_images
labels = train_normal_labels + train_malade_labels + test_normal_labels + test_malade_labels

# Convertir les listes en tableaux numpy
# à quoi sert numpy: manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.
X = np.array(images)
y = np.array(labels)

# Aplatir les images pour les rendre compatibles avec l'algorithme SVM
X = X.reshape((X.shape[0], -1))

# Standardiser les données
# definition de la standardisation: mettre les valeurs des variables pour qu’elles se situent dans un intervalle spécifié
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer les jeux de données en entraînement et test
# On sépare le jeu de données en mettant 80% des données dans le jeu de train et 20% pour le test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, stratify=y, random_state=42)

# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à tester pour GridSearchCV
param_grid = {
    'C': [30],
    'kernel': ['rbf'],
    'gamma': ['scale']
}
# C : paramètre de régularisation, il sert à maximiser la marge et minimiser l'erreur de classification
# Kernel: type de fonction noyau utilisé, rbf dans mon cas "radial basis function"
# Gamma : coefficient pour les noyaux, un gamma faible signifie une influence large, un gamma élevé signifie une influence étroite
#                                      fonction gaussienne plus plate/ plus pointue

# Initialiser GridSearchCV avec le modèle SVM et les paramètres en intégrant la cross-validation
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')

# Entraîner GridSearchCV
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés pour ajuster
print("Best parameters found: ", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Afficher les résultats
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Calculer et afficher la matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Afficher la matrice de confusion sous forme de plot
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = np.unique(y)
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2 else "black")

plt.tight_layout()
plt.show()

Best parameters found:  {'C': 30, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.9587628865979382
Classification Report:
               precision    recall  f1-score   support

      malade       0.97      0.97      0.97      1397
      normal       0.92      0.93      0.93       543

    accuracy                           0.96      1940
   macro avg       0.95      0.95      0.95      1940
weighted avg       0.96      0.96      0.96      1940



C: 30  
Gamma: scale
Kernel: rbf

Accuracy: 96% des prédictions du modèle sur l'ensemble du jeu de test étaient correctes

Malade

    precision : 0.98. Proportion de vrais positifs parmi les exemples classés comme "malade".
    recall : 0.97. Proportion de vrais positifs parmi tous les exemples réellement "malade".
    f1-score : 0.98. Moyenne harmonique de la précision et du rappel, fournissant un seul score pour évaluer le modèle.
    support : 931. Nombre de vrais exemples de la classe "malade" dans l'ensemble de test.

Normal

    precision : 0.93. Proportion de vrais positifs parmi les exemples classés comme "normal".
    recall : 0.94. Proportion de vrais positifs parmi tous les exemples réellement "normal".
    f1-score : 0.94. Moyenne harmonique de la précision et du rappel pour la classe "normal".
    support : 362. Nombre de vrais exemples de la classe "normal" dans l'ensemble de test.

In [3]:
# Charger les images de chaque classe pour l'entraînement et les tests
train_normal_images = load_images_from_folder('./data/chest_Xray/train/NORMAL')
train_malade_images = load_images_from_folder('./data/chest_Xray/train/PNEUMONIA')
test_normal_images = load_images_from_folder('./data/chest_Xray/test/NORMAL')
test_malade_images = load_images_from_folder('./data/chest_Xray/test/PNEUMONIA')

# Créer les labels correspondants
train_normal_labels = ['normal'] * len(train_normal_images)
train_malade_labels = ['malade'] * len(train_malade_images)
test_normal_labels = ['normal'] * len(test_normal_images)
test_malade_labels = ['malade'] * len(test_malade_images)

# Combiner les images et les labels
images = train_normal_images + train_malade_images + test_normal_images + test_malade_images
labels = train_normal_labels + train_malade_labels + test_normal_labels + test_malade_labels

# Convertir les listes en tableaux numpy
X = np.array(images)
y = np.array(labels)

# Aplatir les images pour les rendre compatibles avec l'algorithme SVM
X = X.reshape((X.shape[0], -1))

# Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer les jeux de données en entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, stratify=y, random_state=42)

# Appliquer SMOTE pour équilibrer les classes dans les données d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Rétablir la forme originale des images
X_train_images = X_train_resampled.reshape((-1, 128, 128, 1))


# Définir les hyperparamètres à tester pour GridSearchCV
param_grid = {
    'C': [30],
    'kernel': ['rbf'],
    'gamma': ['scale']
}

# Initialiser GridSearchCV avec le modèle SVM et les paramètres en intégrant la cross-validation
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')

# Entraîner GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Afficher les meilleurs hyperparamètres trouvés pour ajuster
print("Best parameters found: ", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé pour faire des prédictions
best_model = grid_search.best_estimator_

# Afficher les résultats sur les données de test
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))